In [25]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn import preprocessing 

train_df = pd.read_csv('./input/train.csv')
test_df = pd.read_csv('./input/test.csv')

In [26]:
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')   
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
guess_ages = np.zeros((2,3))
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)
freq_port = train_df.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)    
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
train_df['Fare'].fillna(train_df['Fare'].dropna().median(), inplace=True)    

In [27]:
X_train = train_df
Y_train = X_train.pop("Survived")
X_test  = test_df.drop("PassengerId", axis=1).copy()

In [28]:
X_TF_train = X_train.as_matrix()
Y_TF_train = Y_train.as_matrix()
Y_TF_train = np.reshape(Y_TF_train, (-1,1))
X_TF_test = X_test.as_matrix()

X_TF_N_train = preprocessing.scale(X_TF_train)
Y_TF_N_train = preprocessing.scale(Y_TF_train)

D:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [29]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[8])]

estimator = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[64,32,8],
    n_classes=2,
    model_dir="./tmp/titanic_model")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":X_TF_train},
    y=Y_TF_train,
    num_epochs=None,
    shuffle=True)

estimator.fit(input_fn=train_input_fn, steps=2000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000000126B5DD8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './tmp/titanic_model'}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:C

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x00000000126B5710>, 'hidden_units': [64, 32, 8], 'feature_columns': (_NumericColumn(key='x', shape=(8,), default_value=None, dtype=tf.float32, normalizer_fn=None),), 'optimizer': None, 'activation_fn': <function relu at 0x000000000850E048>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [30]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":X_TF_train},
    y=Y_TF_train,
    num_epochs=1,
    shuffle=False)

accuracy_score = estimator.evaluate(input_fn=eval_input_fn)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-11-17-08:40:14
INFO:tensorflow:Restoring parameters from ./tmp/titanic_model\model.ckpt-26000
INFO:tensorflow:Finished evaluation at 2017-11-17-08:40:16
INFO:tensorflow:Saving dict for global step 26000: accuracy = 0.915825, accuracy/baseline_label_mean = 0.383838, accuracy/threshold_0.500000_mean = 0.915825, auc = 0.964478, auc_precision_recall = 0.956915, global_step = 26000, labels/actual_label_mean = 0.383838, labels/prediction_mean = 0.384368, loss = 0.217418, precision/positive_threshold_0.500000_mean = 0.92381, recall/positive_threshold_0.500000_mean = 0.850877


In [42]:
tmp_y = np.zeros((418,))
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":X_TF_test},
    y=tmp_y,
    batch_size=418,
    num_epochs=1,
    shuffle=False)
pred = estimator.predict_classes(input_fn=test_input_fn,as_iterable=False)
print(pred.size)
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": pred
    })
submission.to_csv('./output/submission.csv', index=False)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from ./tmp/titanic_model\model.ckpt-26000
418
